一个图中的三角形个数(不区分边的方向)，可以在一定程度上反映该图的联通性，我们可以先通过两次简单的aggerateMessages得到每个节点的三角形统计值，流程如下：   
![avatar](./pic/三角形计数.png)

图1表示初始图，每个节点的标识符表示为a,b,c,   

1）第一次aggerateMessages：首先各节点将其标识符发送给它的邻居节点，每个节点收到邻居收到邻居节点发送来的消息，将其与自己的标识符拼接为一个列表，如a节点，拼接后为[[a,b],[a,c]],得到图2,   
2）第二次aggerateMessages：然后继续，将当前的list发送给邻居，比如a节点将[[a,b],[a,c]]发送给邻居节点b，b接受到a发来的list之后，将自己的列表[[b,a],[b,c]]与其合并，合并方式为：依次选出每个节点列表中的元素(这里也是list),进行俩俩组合，组合规则为：1. 首尾一致才凭借，比如[b,a]可以和[a,c]拼接为[b,a,c]，但[b,c]不可以与[a,b]拼接，2. 如果拼接后的三元组的首个标识和最后一个标识一样，需要过滤掉，比如[b,a]与[a,b]拼接后得到[b,a,b]，这种需要过滤掉，最后得到图3的结果  
3）最后统计三角形个数，对于每个节点，如果同时出现了形如[a,b,c]和[a,c,b]这样的三元组，说明该节点处于三角形(a,b,c)中

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@9766443
sc = org.apache.spark.SparkContext@2ad4a277


org.apache.spark.SparkContext@2ad4a277

In [2]:
val g=util.GraphGenerators.rmatGraph(sc,32,64).removeSelfEdges()
val newEdges=g.edges.flatMap((edge)=>Array(Edge(edge.srcId,edge.dstId,edge.attr),Edge(edge.dstId,edge.srcId,edge.attr)))
val initGraph=Graph(g.vertices,newEdges)

g = org.apache.spark.graphx.impl.GraphImpl@58ae0757
newEdges = MapPartitionsRDD[24] at flatMap at <console>:32
initGraph = org.apache.spark.graphx.impl.GraphImpl@3caaf92e


org.apache.spark.graphx.impl.GraphImpl@3caaf92e

In [3]:
//第一次aggregateMessage
val secRDD=initGraph.aggregateMessages[Array[VertexId]](
    sendMsg=(edge)=>edge.sendToDst(Array(edge.srcId)),
    mergeMsg=(a,b)=>a++b //列表拼接用俩++
)

secRDD = VertexRDDImpl[43] at RDD at VertexRDD.scala:57


VertexRDDImpl[43] at RDD at VertexRDD.scala:57

In [4]:
secRDD.take(1)  //这里为(a,[b,c,d])的形式

Array((16,Array(18, 20, 21, 22, 24, 28, 17, 20)))

In [5]:
//将(a,[b,c,d])的形式转换为[(a,b),(a,c),(a,d)]的格式
val secVerticesRDD=secRDD.map(x=>{
    val vi=x._1
    val vil=x._2
    var combineArray=Array[Tuple2[VertexId,VertexId]]()
    for(nvi<-vil){
        if(vi!=nvi && !combineArray.contains((vi,nvi))) combineArray=combineArray:+(vi,nvi)
    }
    (vi,combineArray)
})

secVerticesRDD = MapPartitionsRDD[44] at map at <console>:32


MapPartitionsRDD[44] at map at <console>:32

In [6]:
secVerticesRDD.take(1)

Array((16,Array((16,18), (16,20), (16,21), (16,22), (16,24), (16,28), (16,17))))

In [7]:
val secGraph=Graph(secVerticesRDD,initGraph.edges)

secGraph = org.apache.spark.graphx.impl.GraphImpl@4724942b


org.apache.spark.graphx.impl.GraphImpl@4724942b

In [8]:
//第二次aggregateMessages
val thrRDD=secGraph.aggregateMessages[Array[Tuple2[VertexId,VertexId]]](
    sendMsg=(edge)=>edge.sendToDst(edge.srcAttr),
    mergeMsg=(a,b)=>a++b
)

thrRDD = VertexRDDImpl[64] at RDD at VertexRDD.scala:57


VertexRDDImpl[64] at RDD at VertexRDD.scala:57

In [9]:
thrRDD.take(1)

Array((16,Array((18,16), (18,19), (18,20), (18,24), (18,17), (18,26), (20,16), (20,18), (20,22), (20,24), (20,30), (20,17), (20,19), (20,21), (20,23), (21,16), (21,17), (21,20), (21,23), (22,16), (22,20), (22,26), (22,31), (22,30), (24,16), (24,17), (24,18), (24,20), (24,25), (24,26), (24,27), (24,28), (24,29), (28,16), (28,29), (28,24), (17,24), (17,25), (17,16), (17,18), (17,19), (17,20), (17,21), (20,16), (20,18), (20,22), (20,24), (20,30), (20,17), (20,19), (20,21), (20,23))))

In [10]:
//将[a,b][b,c]拼接为[a,b,c]
def expand(org:Array[Tuple2[VertexId,VertexId]],tgt:Array[Tuple2[VertexId,VertexId]]):Array[Tuple3[VertexId,VertexId,VertexId]]={
    var combineArray=Array[Tuple3[VertexId,VertexId,VertexId]]()
    for(i<-org){
        for(j<-tgt){
            if(i._2==j._1 && i._1!=j._2 && !combineArray.contains((i._1,i._2,j._2))) combineArray=combineArray:+(i._1,i._2,j._2)
        }
    }
    combineArray
}
var expandRDD=secGraph.vertices.join(thrRDD).map(x=>(x._1,expand(x._2._1,x._2._2)))

expandRDD = MapPartitionsRDD[68] at map at <console>:43


expand: (org: Array[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId)], tgt: Array[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId)])Array[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId)]


MapPartitionsRDD[68] at map at <console>:43

In [11]:
expandRDD.take(1)

Array((16,Array((16,18,19), (16,18,20), (16,18,24), (16,18,17), (16,18,26), (16,20,18), (16,20,22), (16,20,24), (16,20,30), (16,20,17), (16,20,19), (16,20,21), (16,20,23), (16,21,17), (16,21,20), (16,21,23), (16,22,20), (16,22,26), (16,22,31), (16,22,30), (16,24,17), (16,24,18), (16,24,20), (16,24,25), (16,24,26), (16,24,27), (16,24,28), (16,24,29), (16,28,29), (16,28,24), (16,17,24), (16,17,25), (16,17,18), (16,17,19), (16,17,20), (16,17,21))))

In [12]:
expandRDD.map(x=>{
    //统计三角形数量
    val vid=x._1
    val arr=x._2
    var cnt=0
    for(i<-arr) if(arr.contains((i._1,i._3,i._2))) cnt=cnt+1
    (vid,cnt/2)
}).take(32)

Array((16,10), (22,3), (28,2), (30,2), (24,12), (18,9), (20,13), (26,4), (19,4), (21,4), (25,3), (29,2), (27,1), (23,2), (17,12), (31,1))

In [13]:
//官方api结果
initGraph.triangleCount().vertices.take(32)

Array((16,10), (22,3), (28,2), (30,2), (24,12), (18,9), (20,13), (26,4), (19,4), (21,4), (25,3), (29,2), (27,1), (23,2), (17,12), (31,1))